In [166]:
import os
import random
import time
from bs4 import BeautifulSoup
import asyncio
import pandas as pd
import regex as re
import string
import json

from playwright.async_api import async_playwright, expect

## Defining functions for each page of the intake form

In [2]:
async def open_browser(headless=False):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.firefox.launch(headless=False)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [3]:
## driver, page = await open_browser()

In [4]:
# url = 'https://www.hims.com/c/wm/introduction'
# await page.goto(url)

In [5]:
## Page 1 - "https://www.hims.com/c/wm/introduction" - "Explore weight loss plans" - click continue

async def click_continue():
    await expect(page.locator('button:text("Continue")')).to_be_enabled()
    await page.locator('button:text("Continue")').click()
    await asyncio.sleep(4)


In [6]:
## Page 2 - "https://www.hims.com/c/wm/your-goal" - "What's your weight loss goal?" - selecting goal

async def loss_goal(goal):

    if goal in range(1, 15):
        goal_path = '//*[@id="your-goals-losing-1-15-label"]'
    elif goal in range(16, 50):
        goal_path = '//*[@id="your-goals-losing-16-50-label"]'
    elif goal in range(51, 300):
        goal_path = '//*[@id="your-goals-losing-51-plus-label"]'
    else:
        goal_path = '//*[@id="your-goals-not-sure-label"]'

    await expect(page.locator(goal_path)).to_be_enabled()
    await page.locator(goal_path).click()
    await asyncio.sleep(4.2)

    return print(f"Chose weight loss goal: {goal} lbs")    

In [7]:
## await loss_goal(14)

In [8]:
## Page 3 "Your goal to lose ___ is closer than you think" - click next

async def click_next():
    await expect(page.locator('button:text("Next")')).to_be_enabled()
    await page.locator('button:text("Next")').click()
    await asyncio.sleep(4.5)

In [9]:
## Page 4 "Lose up to 20% of your weight" - click next

# async def click_next():
#     await expect(page.locator('button:text("Next")')).to_be_enabled()
#     await page.locator('button:text("Next")').click()
#     await asyncio.sleep(4.5)

In [10]:
## Page 5 "Select the state you live in"  - Creating function to select the proper state from the dropdown menu and proceed to next page

async def key_down(state):
  
    state_list = ['AA', 'AE', 'Alabama', 'Alaska', 'AP', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyomin']
    if state in state_list:        
        times = state_list.index(state)
        count = 0
        while count <= times:
            await page.keyboard.press("ArrowDown")
            count = count + 1
        await page.keyboard.press("Enter")

    else:
        print('State name entry error')

async def select_state(state):
    ## opening the dropdown menu of states
    xpath_state_menu = '//*[@id="state-select"]'
    await page.locator(xpath_state_menu).click(delay=4000)
    ## selecting the desired state from the dropdown menu (see above function)
    await key_down(state)
    await asyncio.sleep(1)
    ## checking the terms and conditions box
    xpath_terms = '//*[@id="termsAndConditions"]'
    await page.locator(xpath_terms).click()
    await asyncio.sleep(1.6)
    ## clicking the continue button
    await expect(page.locator('button:text("Continue")')).to_be_enabled()
    await page.locator('button:text("Continue")').click()
    await asyncio.sleep(3)
    
    return print(f'Selected state: {state}')



In [11]:
## await select_state('Minnesota')

In [12]:
## Page 6 "To verify eligibility, tell us your date of birth" - Entering birthdate (must be in MM-DD-YYYY format)

async def fill_birthday(dob):
    xpath_bd_form = page.locator('//*[@id="dob"]')
    await xpath_bd_form.fill(dob)
    await asyncio.sleep(1.4)
    await expect(page.locator('button:text("Continue")')).to_be_enabled()
    await page.locator('button:text("Continue")').click()
    await asyncio.sleep(2)

    return print(f'Entered birthday: {dob}')
    
    

In [13]:
## await fill_birthday('01-01-1993')

In [14]:
## Page 7 "Nice! Looks like you're good to go on our end" - click next

# async def click_next():
#     await expect(page.locator('button:text("Next")')).to_be_enabled()
#     await page.locator('button:text("Next")').click()
#     await asyncio.sleep(4.5)

In [15]:
## Page 8 "How many different weight loss programs have you tried in the past?" - select number of programs

async def weight_programs(diet_number):

    if diet_number == "None, this is my first time trying":
        try:
            await page.locator('label:has-text("this is my first time trying")').click();
        except:
            await page.locator('label:has-text("This is my first time exploring options")').click();
    elif diet_number == "1-5":
        try:
            await page.locator('label:has-text("1-5")').click();
        except:
            await page.locator('label:has-text("tried a few things, ready to step it up")').click();
    elif diet_number == "6-10":
        try:
            await page.locator('label:has-text("6-10")').click();
        except:
            await page.locator('label:has-text("tried a few things, ready to step it up")').click();
    elif diet_number == "More than I can remember":
        try:
            await page.locator('label:has-text("More than I can remember")').click();
        except:
            await page.locator('label:has-text("tried everything, nothing has worked")').click();

    await asyncio.sleep(2.4)

    return print(f"experience with weight programs: {diet_number}") 

In [16]:
## await weight_programs(3)

In [17]:
## Page 9 "Do you hae any family members who struggle with their weight? - click yes, no, or not sure

async def family_struggles(answer):

    if answer == 'Yes':
        family_path = '//*[@id="any-family-with-weight-struggle-yes-label"]'
    elif answer == 'No':
        family_path = '//*[@id="any-family-with-weight-struggle-no-label"]'
    else:
        family_path = '//*[@id="any-family-with-weight-struggle-not-sure-label"]'

    await expect(page.locator(family_path)).to_be_enabled()
    await page.locator(family_path).click()
    await asyncio.sleep(4.1)

    return print(f"Chose family weight loss history: {answer} programs") 

In [18]:
## await family_struggles('No')

In [19]:
## Page 10 - Click next

# await click_next()

In [20]:
## Page 11 "How would you describe the level of stress you experience in your daily life"

async def daily_stress(amount):

    if amount == 'I rarely feel stressed':
        stress_path = '//*[@id="daily-life-stress-level-rarely-stressed-label"]'
    elif amount == 'I feel stressed a few times a week':
        stress_path = '//*[@id="daily-life-stress-level-few-times-a-week-label"]'
    elif amount == 'I feel stressed all the time':
        stress_path = '//*[@id="daily-life-stress-level-all-the-time-label"]'

    await expect(page.locator(stress_path)).to_be_enabled()
    await page.locator(stress_path).click()
    await asyncio.sleep(4.4)

    return print(f"Chose stress level: {amount}") 

In [21]:
## await daily_stress('I feel stressed a few times a week')

In [22]:
## Page 12 "On average, how much sleep do you get a night" - select amount

async def nightly_sleep(hours):

    if hours > 9:
        sleep_path = '//*[@id="quality-sleep-more-than-9hrs-label"]'
    elif hours in range(7, 9):
        sleep_path = '//*[@id="quality-sleep-7-9hrs-label"]'
    elif hours < 7:
        sleep_path = '//*[@id="quality-sleep-less-than-7hrs-label"]'
    else:
        sleep_path = '//*[@id="quality-sleep-varies-with-trouble-sleeping-label"]'

    await expect(page.locator(sleep_path)).to_be_enabled()
    await page.locator(sleep_path).click()
    await asyncio.sleep(4.4)

    return print(f"Chose nightly sleep amount: {hours} hours") 

In [23]:
## await nightly_sleep(7)

In [24]:
## Page 13 - Click next

## await click_next()

In [25]:
## Page 14 "Where do you hold most of your weight?" - Choose body area

async def weight_place(area):

    if area == 'Around my stomach or waist':
        weight_path = '//*[@id="weight-hold-sites-waist-or-stomach-label"]'
    elif area == 'Hips and thighs':
        weight_path = '//*[@id="weight-hold-sites-hips-and-thighs-label"]'
    elif area == 'All over':
        weight_path = '//*[@id="weight-hold-sites-all-over-label"]'

    await expect(page.locator(weight_path)).to_be_enabled()
    await page.locator(weight_path).click()
    await asyncio.sleep(4.6)

    return print(f"Chose weight area: {area}") 

In [26]:
## await weight_place('Hips and thighs')

In [27]:
## Page 15 - Click next

## await click_next()

In [28]:
## Page 16 - select type of food craving

async def craving_type(craving):

    if craving == 'Sweet':
        craving_path = '//*[@id="cravings-sweet-label"]'
    elif craving == 'Salty':
        craving_path = '//*[@id="cravings-salty-label"]'
    elif craving == 'Both':
        craving_path = '//*[@id="cravings-both-label"]'
    elif craving == "I don't have cravings":
        craving_path = '//*[@id="cravings-no-cravings-label"]'

    await expect(page.locator(craving_path)).to_be_enabled()
    await page.locator(craving_path).click()
    await asyncio.sleep(2.8)

    return print(f"Chose craving type: {craving}") 

In [29]:
## await craving_type('Salty')

In [30]:
## Page 17 "Which eating habit sounds most like you" - select one

async def eating_habit(habit):

    if habit == "I'm usually multitasking when I eat":
        habit_path = '//*[@id="automatic-label"]'
    elif habit == "I tend to snack when I'm stressed":
        habit_path = '//*[@id="anxious-label"]'
    elif habit == 'I turn to food for comfort':
        habit_path = '//*[@id="escapist-label"]'
    elif habit == 'I often reward myself with food':
        habit_path = '//*[@id="uplifting-label"]'
    elif habit == 'I center my social life around dining out':
        habit_path = '//*[@id="enthusiastic-label"]'

    await expect(page.locator(habit_path)).to_be_enabled()
    await page.locator(habit_path).click()
    await asyncio.sleep(2.3)
    await expect(page.locator('button:text("Continue")')).to_be_enabled()
    await page.locator('button:text("Continue")').click()
    await asyncio.sleep(4)

    return print(f"Chose food habit type: {habit}") 

In [31]:
## await eating_habit("I tend to snack when I'm stressed")

In [32]:
## Page 18 "Any additional areas you'd like your program to focus on?" - select one

async def other_focus(focus):

    if focus == "Realistic nutrition plan":
        focus_path = '//*[@id="nutrition-label"]'
    elif focus == "Movement that works for you":
        focus_path = '//*[@id="exercise-label"]'
    elif focus == 'Getting better sleep':
        focus_path = '//*[@id="sleep-label"]'
    elif focus == 'Building healthier habits':
        focus_path = '//*[@id="habits-label"]'
    elif focus == 'All of the above':
        focus_path = '//*[@id="all-label"]'

    await expect(page.locator(focus_path)).to_be_enabled()
    await page.locator(focus_path).click()
    await asyncio.sleep(2.3)
    await expect(page.locator('button:text("Continue")')).to_be_enabled()
    await page.locator('button:text("Continue")').click()
    await asyncio.sleep(4.3)

    return print(f"Chose other program focus: {focus}") 

In [33]:
## await other_focus('Building healthier habits')

In [34]:
## Page 19 "What would reaching your goal weight mean for you" - select one

async def goal_meaning(meaning):

    if meaning == "Having more energy":
        meaning_path = '//*[@id="energy-label"]'
    elif meaning == "Feeling more confident":
        meaning_path = '//*[@id="confident-label"]'
    elif meaning == 'Improving overall health':
        meaning_path = '//*[@id="overall-health-label"]'
    elif meaning == 'Feeling better in my body':
        meaning_path = '//*[@id="better-body-label"]'
    elif meaning == 'Feeling good in clothes':
        meaning_path = '//*[@id="clothes-label"]'

    await expect(page.locator(meaning_path)).to_be_enabled()
    await page.locator(meaning_path).click()
    await asyncio.sleep(2.8)
    await expect(page.locator('button:text("Continue")')).to_be_enabled()
    await page.locator('button:text("Continue")').click()
    await asyncio.sleep(4)

    return print(f"Chose what reaching goal would mean: {meaning}") 

In [35]:
## await goal_meaning("Having more energy")

In [36]:
## Page 20 "Now let's find which treatments you're elgible for - enter email

def email_generator(size=10, chars=string.ascii_lowercase + string.digits):
    alias = ''.join(random.choice(chars) for _ in range(size))
    return alias+'@yahoo.com'

def password_generator(size=10, chars=string.ascii_lowercase + string.digits):
    password = ''.join(random.choice(chars) for _ in range(size))
    return password

async def enter_email():
    email_form = page.locator('//*[@id="email"]')
    email = email_generator()
    await email_form.fill(email)
    await asyncio.sleep(1)
    await click_continue()
    await asyncio.sleep(2)

    return print(f'Entered email address: {email}')

async def enter_password():
    password_form = page.locator('//*[@id="password"]')
    password = password_generator()
    await password_form.fill(password)
    await asyncio.sleep(1)
    await click_continue()
    await asyncio.sleep(2.5)

    return print(f'Entered password: {password}')


In [37]:
## await enter_email()
## await enter_password()

In [38]:
## Page 22 "What is your height/weight?" - enter height and weight

async def enter_height(feet, inches):
    feet_form = page.locator('//*[@id="feet"]')
    await feet_form.fill(feet)
    await asyncio.sleep(1.6)
    inches_form = page.locator('//*[@id="inches"]')
    await inches_form.fill(inches)
    await asyncio.sleep(1.8)

    return print(f'Entered height: {feet} feet {inches} inches')

async def enter_weight(pounds):
    pounds_form = page.locator('//*[@id="pounds"]')
    await pounds_form.fill(pounds)
    await asyncio.sleep(2)

    return print(f'Entered weight: {pounds} lbs')


    

In [39]:
# await enter_height('6', '2')
# await enter_weight('199')
# await click_next()

In [40]:
## Page 23 "What is your goal weight?" - enter weight

async def target_weight(target):
    target_form = page.locator('//input[@id="3f149c5d-bc24-42b3-b381-ce5556454c06"]')
    await target_form.fill(target)
    await asyncio.sleep(2.3)
    await click_continue()

    return print(f'Chose target weight: {target}')
    

In [41]:
## await target_weight('170')

In [42]:
## Page 24 "Is your current weight the most you've ever weighed?" - answer yes or no

async def most_weighed(most):
    
    if most == 'Yes':
        most_path = '//*[@id="0714e1a4-031e-496b-9478-506c9ce6e628-648dc58b-11d7-4166-95d1-8c0df29e08ce-label"]'
    elif most == 'No':
        most_path = '//*[@id="0714e1a4-031e-496b-9478-506c9ce6e628-0ff405be-cca4-43d3-ab86-51b0e5cd9ff9-label"]'

    await expect(page.locator(most_path)).to_be_enabled()
    await page.locator(most_path).click()
    await asyncio.sleep(2.8)

    return print(f'Is at highest ever weight: {most}')

In [43]:
## await most_weighed('Yes')

In [44]:
## Page 25 "What was your sex assigned at birth

async def birth_sex(sex):

    if sex == 'Male':
        sex_path = '//*[@id="5ad4cf8e-3c37-4c2f-a15a-7d3d78bb961f-743c1760-45c8-4f1c-919b-b0a4a1366e33-label"]'
    elif sex == 'Female':
        sex_path = '//*[@id="5ad4cf8e-3c37-4c2f-a15a-7d3d78bb961f-b03ca92a-2a11-435f-b518-95507d1ef46a-label"]'

    await expect(page.locator(sex_path)).to_be_enabled()
    await page.locator(sex_path).click()
    await asyncio.sleep(2.8)

    return print(f'Chose sex at birth: {sex}')

In [45]:
## await birth_sex('Female')

In [46]:
## Page 25a "Are you currently pregnant or trying to become pregnat?" - answer

async def pregnant_status(pregnant):
    
    if pregnant == 'No':
        pregnant_path = '//*[@id="c6b7fd8b-43f0-4c42-add1-6afde8074e79-9e54d5b1-a320-4aa0-b9e9-4ba83060a9e0-label"]'
    elif pregnant == 'Yes':
        pregnant_path = '//*[@id="c6b7fd8b-43f0-4c42-add1-6afde8074e79-68499002-c9eb-456a-9d4f-bcfa4c4e9449-label"]'
    elif pregnant == 'Trying':
        pregnant_path = '//*[@id="c6b7fd8b-43f0-4c42-add1-6afde8074e79-5fc0a0c3-85c5-445d-8b25-896d7b23b224-label"]'

    await expect(page.locator(pregnant_path)).to_be_enabled()
    await page.locator(pregnant_path).click()
    await asyncio.sleep(2.8)

    return print(f'Chose pregnancy status: {pregnant}')

In [47]:
## await pregnant_status('Yes')

In [48]:
## Page 25b "Are you currently pregnant or trying to become pregnat?" - answer

async def bf_status(breastfeeding):
    
    if breastfeeding == 'No':
        bf_path = '//*[@id="22b6d502-7ed0-4ff8-92ab-11865f49fc62-d7e6c41c-f8c5-4c4c-81a0-6d9f6cdcac7a-label"]'
    elif breastfeeding == 'Yes':
        bf_path = '//*[@id="22b6d502-7ed0-4ff8-92ab-11865f49fc62-bd3ea106-e5d5-4ef7-b63e-648505e7deb4-label"]'

    await expect(page.locator(bf_path)).to_be_enabled()
    await page.locator(bf_path).click()
    await asyncio.sleep(2.8)

    return print(f'Chose breastfeeding status: {breastfeeding}')

In [49]:
## await bf_status('Yes')

In [50]:
## Page 26 "Do you identify as a man?" - yes or no

async def male_identity(identity):
    
    if identity == 'Yes':
        male_path = '//*[@id="f27b7009-bf9d-4e5c-b5f7-a35f10199e8b-42e9446a-906f-44c5-97cf-72f9d9c16985-label"]'
    elif identity == 'No':
        male_path = '//*[@id="f27b7009-bf9d-4e5c-b5f7-a35f10199e8b-ddbe04d8-8bf6-4b00-9904-39481201e9c5-label"]'

    await expect(page.locator(male_path)).to_be_enabled()
    await page.locator(male_path).click()
    await asyncio.sleep(3)

    return print(f'Identify as man: {identity}')

In [51]:
## await male_identity('Yes')

In [52]:
## Page 26a "What gender do you most identify with?" - choose option

async def gender_choice(gender):

    if gender == "Man":
        gender_path = '//*[@id="0b25c3a2-ddb1-427c-9b61-5c595f412cab-25eaab3f-2a97-4fef-b26a-e457e24c1186-label"]'
    elif gender == "Woman":
        gender_path = '//*[@id="0b25c3a2-ddb1-427c-9b61-5c595f412cab-bce8e927-5682-477d-9728-c433d7c53624-label"]'
    elif gender == 'Genderqueer':
        gender_path = '//*[@id="0b25c3a2-ddb1-427c-9b61-5c595f412cab-94502904-b129-4158-a9f4-1efbf0b1a2ca-label"]'
    elif gender == 'Non-binary':
        gender_path = '//*[@id="0b25c3a2-ddb1-427c-9b61-5c595f412cab-47304175-2e56-447d-b4ee-6074a372a462-label"]'
    elif gender == 'Agender':
        gender_path = '//*[@id="0b25c3a2-ddb1-427c-9b61-5c595f412cab-ae802ba6-e5df-4d14-9339-b2d90cf44e61-label"]'
    elif gender == 'Questioning':
        gender_path = '//*[@id="0b25c3a2-ddb1-427c-9b61-5c595f412cab-b1d963fa-db27-4844-b359-409ee8479187-label"]'

    await expect(page.locator(gender_path)).to_be_enabled()
    await page.locator(gender_path).click()
    await asyncio.sleep(2.5)

    return print(f"Chose gender: {gender}") 

In [53]:
## await gender_choice('Genderqueer')

In [54]:
## Page 27 "Have you ever experienced any of these symptoms?" - choose option

async def eating_disorder(disorder):

    if disorder == "Causing yourself to vomit in order to lose weight":
        disorder_path = '//*[@id="705fa680-a46b-4987-8960-82cfa1d815ea-label"]'
    elif disorder == "Frequently eating very large amounts of food and feeling like you can't stop eating":
        disorder_path = '//*[@id="680d8d8f-a32f-4073-b809-b2c1256d4344-label"]'
    elif disorder == 'Severely limiting the amount of food you eat due to an intense fear of gaining weight':
        disorder_path = '//*[@id="e783faf4-f18d-4543-8521-11b6e392994b-label"]'
    elif disorder == 'None of these':
        disorder_path = '//*[@id="41765552-942d-438c-82be-78644f237c1d-label"]'
    
    await expect(page.locator(disorder_path)).to_be_enabled()
    await page.locator(disorder_path).click()
    await asyncio.sleep(1)
    await click_continue()

    return print(f"Chose eating disorder symptoms: {disorder}") 

In [55]:
## await eating_disorder('Causing yourself to vomit in order to lose weight')

In [56]:
## Page 27a "Have you been diagnosed with any of the following conditions?" - choose option

async def disorder_diagnosis(diagnosis):

    if diagnosis == "Anorexia":
        diagnosis_path = '//*[@id="fd9a523c-7f2c-47de-82d9-9bff51c4dd79-label"]'
    elif diagnosis == "Bulimia":
        diagnosis_path = '//*[@id="cb12f86f-d4e4-4c65-9acc-dbddce9619bb-label"]'
    elif diagnosis == 'Binge eating disorder':
        diagnosis_path = '//*[@id="1e4b6ee8-3784-4c61-92c6-b02b68cebfb7-label"]'
    elif diagnosis == 'None of these':
        diagnosis_path = '//*[@id="7068ce16-bc03-41b4-933a-6390bac62cab-label"]'
    
    await expect(page.locator(diagnosis_path)).to_be_enabled()
    await page.locator(diagnosis_path).click()
    await asyncio.sleep(1.2)
    await click_continue()

    return print(f"Chose eating disorder diagnosis: {diagnosis}") 

In [57]:
## await disorder_diagnosis('Anorexia')

In [58]:
## Page 27b "Have you been in remission from your anorexia or bulimia eating disorder for one year or more?" - pick answer

async def remission_choice(remission):

    if remission == "No, I am currently being treated":
        await page.locator('label:has-text("No, I am currently being treated")').click(); 
    elif remission == "No, I have been in remission for less than one year":
        await page.locator('label:has-text("No, I have been in remission for less than one year")').click();
    elif remission == 'Yes, I have been in remission for one year or more':
       await page.locator('label:has-text("Yes, I have been in remission for one year or more")').click();

    await asyncio.sleep(2.2)

    return print(f"Chose remission status: {remission}") 

In [59]:
## await remission_choice('Yes, I have been in remission for one year or more')

In [60]:
## Page 27c "Have you purged or forced yourself to vomit in order to lose weight within the last 12 months?" - yes or no

async def purge_choice(purge):

    if purge == "No":
        await page.locator('label:has-text("No")').click();
        ## purge_path = '//*[@id="d31b9d37-9aa6-444f-9efe-bb7a2f04f66e-aeace109-f688-43ab-8a02-9828b15ee8a7-label"]'
    elif purge == "Yes":
        await page.locator('label:has-text("Yes")').click();
        ## purge_path = '//*[@id="d31b9d37-9aa6-444f-9efe-bb7a2f04f66e-a2a25735-45f3-44e0-8cf8-79520db1c42b-label"]'

    # await expect(page.locator(purge_path)).to_be_enabled()
    # await page.locator(purge_path).click()
    await asyncio.sleep(2.2)

    return print(f"Have purged in last 12 months: {purge}") 

In [61]:
## await purge_choice('Yes')

In [62]:
## Page 27c-1 "How frequently have you purged or vomited to lose weight over the last 3 months?" - choose answer

async def purge_frequency(frequency):

    if frequency == "About once a week":
        await page.locator('label:has-text("About once a week")').click();
        ## frequency_path = '//*[@id="55e621aa-8f93-4a11-836c-32e95b85fca4-4e919644-65df-4141-8a4b-4dfada65a1b7-label"]'
    elif frequency == "About once a month":
         await page.locator('label:has-text("About once a month")').click();
        ## frequency_path = '//*[@id="55e621aa-8f93-4a11-836c-32e95b85fca4-71434011-402f-4de2-9524-851e052209cd-label"]'
    elif frequency == 'Not at all':
        await page.locator('label:has-text("Not at all")').click();
        ## frequency_path = '//*[@id="55e621aa-8f93-4a11-836c-32e95b85fca4-779f6ea2-5884-4bcd-8f7e-1c6c77bb9567-label"]'

    # await expect(page.locator(frequency_path)).to_be_enabled()
    # await page.locator(frequency_path).click()
    await asyncio.sleep(2.2)

    return print(f"Chose purge frequency: {frequency}") 

In [63]:
## await purge_frequency('About once a week')

In [64]:
## page 27c-2 "Please be aware that purging..." - click 'I understand'

async def purge_disclosure():
    understand_path = '//*[@id="93f51a64-a483-43bd-98bd-e1260330982e-724650de-2ac1-4fdb-a159-e0b52faeeb11-label"]'
    await expect(page.locator(understand_path)).to_be_enabled()
    await page.locator(understand_path).click()
    await asyncio.sleep(2.8)

In [65]:
## await purge_disclosure()

In [66]:
## page 27b-1 "Are you currently receiving psycotherapy treatment?" - yes or no

async def psycotherapy_choice(psycotherapy):
    
    if psycotherapy == 'Yes':
        psycotherapy_path = '//*[@id="2c23b1a5-b19f-4669-8451-fc674c09c088-2aa2453a-6c19-4ce9-8065-34c8ba9b9925-label"]'
    elif psycotherapy == 'No':
        psycotherapy_path = '//*[@id="2c23b1a5-b19f-4669-8451-fc674c09c088-2e00d30b-ea63-489d-baee-b0d87fe15647-label"]'

    await expect(page.locator(psycotherapy_path)).to_be_enabled()
    await page.locator(psycotherapy_path).click()
    await asyncio.sleep(3)

    return print(f'Currently in psycotherapy: {psycotherapy}')

In [67]:
## await psycotherapy_choice('Yes')

In [68]:
async def mental_diagnosis(mental):

    if mental == "No":
        await page.locator('label:has-text("No")').click();
    if mental == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2.3)

    return print(f'Have mental health diagnosis: {mental}')

In [69]:
## await mental_diagnosis('No')

In [70]:
async def mental_condition(condition):

    if condition == "Depression":
        await page.locator('//*[@id="d0817c1c-b7ff-4181-9b2d-984365bb4d7c-label"]').click();
    elif condition == "Generalized anxiety":
        await page.locator('label:has-text("Generalized anxiety")').click();
    elif condition == "Bipolar disease (manic depression)":
        await page.locator('label:has-text("Bipolar disease (manic depression)")').click();
    elif condition == "Panic attack":
        await page.locator('label:has-text("Panic attack")').click();
    elif condition == "Psychiatric hospitalization within the last 3 months":
        await page.locator('label:has-text("Psychiatric hospitalization within the last 3 months")').click();
    elif condition == "Schizophrenia or schizoaffective disorder":
        await page.locator('label:has-text("Schizophrenia or schizoaffective disorder")').click();
    elif condition == "Psychosis":
        await page.locator('label:has-text("Psychosis")').click();
    elif condition == "Borderline personality disorder":
        await page.locator('label:has-text("Borderline personality disorder")').click();

    await asyncio.sleep(.8)
    await click_continue()

    return print(f'Chose mental condition: {condition}')

In [71]:
## await mental_condition("Depression")

In [72]:
async def mental_medication(medication):

    if medication == "No":
        await page.locator('label:has-text("No")').click();
    if medication == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2.3)

    return print(f'Taking mental health medication: {medication}')

In [73]:
## await mental_medication('Yes')

In [74]:
async def choose_med(medlist):

    target_form = page.locator('//*[@id="searchTerm"]')
    await target_form.fill(medlist)
    await asyncio.sleep(1)
    await page.get_by_role("button").nth(1).click()
    await asyncio.sleep(2)
    await click_continue()

    return print(f'Chose medication: {medlist}')
    

In [75]:
## await choose_med('Sertraline')

In [76]:
async def suicidal_choice(suicidal):

    if suicidal == "No":
        await page.locator('label:has-text("No")').click();
    if suicidal == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Having thoughts of suicide: {suicidal}')

In [77]:
## await suicidal_choice('Yes')

In [78]:
async def suicide_disclosure():

    await page.locator('label:has-text("I understand")').click();
    await asyncio.sleep(2.2)

In [79]:
## await suicide_disclosure()

In [80]:
async def chronic_choice(chronic):

    if chronic == "No":
        await page.locator('label:has-text("No")').click();
    if chronic == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Have medical or chronic diseases: {chronic}')

In [81]:
## await chronic_choice('Yes')

In [82]:
async def kidney_choice(kidney):

    if kidney == "Acute or chronic kidney disease (CKD)":
        await page.locator('label:has-text("Acute or chronic kidney disease (CKD)")').click(); 
    elif kidney == "Kidney stones":
        await page.locator('label:has-text("Kidney stones")').click();
    elif kidney == 'No, none of these':
       await page.locator('label:has-text("No, none of these")').click();
    await asyncio.sleep(.5)
    await click_continue()
    await asyncio.sleep(2)

    return print(f"Chose kidney condition: {kidney}") 

In [83]:
## await kidney_choice('Kidney stones')

In [84]:
async def medical_condition(medical):

    if medical == "Hypertension (high blood pressure)":
        await page.locator('label:has-text("Hypertension (high blood pressure)")').click(); 
    elif medical == "Hypotension (low blood pressure)":
        await page.locator('label:has-text("Hypotension (low blood pressure)")').click();
    elif medical == 'Diabetes (Type 1)':
       await page.locator('label:has-text("Diabetes (Type 1)")').click();
    elif medical == 'Diabetes (Type 2)':
       await page.locator('label:has-text("Diabetes (Type 2)")').click();
    elif medical == 'Prediabetes':
       await page.locator('label:has-text("Prediabetes")').click();
    elif medical == 'Seizure disorder (including Epilepsy)':
       await page.locator('label:has-text("Seizure disorder (including Epilepsy)")').click();
    elif medical == 'Heart attack':
       await page.locator('label:has-text("Heart attack")').click();
    elif medical == 'Heart disease':
       await page.locator('label:has-text("Heart disease")').click();
    elif medical == 'Stroke, mini stroke, or TIA':
       await page.locator('label:has-text("Stroke, mini stroke, or TIA")').click();
    elif medical == 'Heart arrhythmia (including long QT syndrome)':
       await page.locator('label:has-text("Heart arrhythmia (including long QT syndrome)")').click();
    elif medical == 'Hyperlipidemia (high cholesterol)':
       await page.locator('label:has-text("Hyperlipidemia (high cholesterol)")').click();
    elif medical == 'Angina':
       await page.locator('label:has-text("Angina")').click();
    elif medical == 'Congestive heart failure':
       await page.locator('label:has-text("Congestive heart failure")').click();
    elif medical == 'Liver issues':
       await page.locator('label:has-text("Liver issues")').click();
    elif medical == 'Cancer':
       await page.locator('label:has-text("Cancer")').click();
    elif medical == 'Glaucoma':
       await page.locator('label:has-text("Glaucoma")').click();
    elif medical == 'Gout':
       await page.locator('label:has-text("Gout")').click();
    elif medical == 'Cystic fibrosis':
       await page.locator('label:has-text("Cystic fibrosis")').click();
    elif medical == 'Hyponatremia':
       await page.locator('label:has-text("Hyponatremia")').click();
    elif medical == 'Phenylketonuria (PKU)':
       await page.locator('label:has-text("Phenylketonuria (PKU)")').click();
    elif medical == 'Obstructive sleep apnea':
       await page.locator('label:has-text("Obstructive sleep apnea")').click();
    elif medical == 'No, none of these apply to me':
       await page.locator('label:has-text("No, none of these apply to me")').click();
        
    await asyncio.sleep(.5)
    await click_continue()
    await asyncio.sleep(2)

    return print(f"Chose medical condition: {medical}") 

In [85]:
## await medical_condition('Diabetes (Type 1)')

In [86]:
async def medical_other(other):

    if other == "No":
        await page.locator('label:has-text("No")').click();
    if other == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Have other medical conditions: {other}')

In [87]:
## await medical_other('No')

In [88]:
async def family_medical(familymed):

    if familymed == "Medullary thyroid cancer":
        await page.locator('label:has-text("Medullary thyroid cancer")').click(); 
    elif familymed == "Multiple endocrine neoplasia type-2":
        await page.locator('label:has-text("Multiple endocrine neoplasia type-2")').click();
    elif familymed == 'Pancreatitis':
       await page.locator('label:has-text("Pancreatitis")').click();
    elif familymed == "Gastroparesis (delayed stomach emptying)":
        await page.locator('label:has-text("Gastroparesis (delayed stomach emptying)")').click();
    elif familymed == 'Diabetes type-2':
       await page.locator('label:has-text("Diabetes type-2")').click();
    elif 'No, none' in familymed:
        await page.locator('label:has-text("No, none")').click();
    await asyncio.sleep(.5)
    await click_continue()
    await asyncio.sleep(2)

    return print(f"Chose family medical history: {familymed}") 

In [89]:
## await family_medical('No, none of these')

In [90]:
async def cardiac_history(cardiac):

    if cardiac == "No":
        await page.locator('label:has-text("No")').click();
    if cardiac == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Has anyone in family died from cardiac event: {cardiac}')

In [91]:
## await cardiac_history('No')

In [92]:
async def surgery_history(surgery):

    if surgery == "No":
        await page.locator('label:has-text("No")').click();
    if surgery == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Have you had surgeries in past: {surgery}')

In [93]:
## await surgery_history('No')

In [94]:
async def glp1_history(glp1):

    if glp1 == "I am currently taking a GLP-1 medication":
        await page.locator('label:has-text("I am currently taking a GLP-1 medication")').click();
    elif glp1 == "I have taken a GLP-1 medication in the past but I'm not currently":
        await page.locator('label:has-text("I have taken a GLP-1 medication in the past but")').click();
    elif glp1 == "I have never taken a GLP-1 medication":
        await page.locator('label:has-text("I have never taken a GLP-1 medication")').click();
    await asyncio.sleep(2)

    return print(f'Have you taken GLP-1 in past: {glp1}')

In [95]:
## await glp1_history("I have taken a GLP-1 medication in the past but I'm not currently")

In [96]:
async def glp1_med(glp1med):

    if "Injectable liraglutide" in glp1med:
        await page.locator('label:has-text("Injectable liraglutide")').click();
    elif "Injectable tirzepatide" in glp1med:
        await page.locator('label:has-text("Injectable tirzepatide")').click();
    elif "Injectable semaglutide" in glp1med:
        await page.locator('label:has-text("Injectable semaglutide")').click();
    elif "Other GLP-1" in glp1med:
        await page.locator('label:has-text("Other GLP-1")').click();
    await asyncio.sleep(.5)
    await click_continue()

    return print(f'What GLP-1 have you taken: {glp1med}')

In [97]:
## await glp1_med('Injectable semaglutide')

In [98]:
async def glp1_last(glp1last):

    if glp1last == "Within the last 2 weeks":
        await page.locator('label:has-text("Within the last 2 weeks")').click();
    elif glp1last == "2-4 weeks ago":
        await page.locator('label:has-text("2-4 weeks ago")').click();
    elif glp1last == "Over 4 weeks ago":
        await page.locator('label:has-text("Over 4 weeks ago")').click();
    await asyncio.sleep(2)

    return print(f'Last took GLP-1: {glp1last}')

In [99]:
## await glp1_last('Within the last 2 weeks')

In [100]:
async def glp1_dose(glp1dose):

    if glp1dose == "0.25 mg":
        await page.locator('label:has-text("0.25 mg")').click();
    elif glp1dose == "0.5 mg":
        await page.locator('label:has-text("0.5 mg")').click();
    elif glp1dose == "1 mg":
        await page.locator('label:has-text("1 mg")').click();
    elif glp1dose == "1.25 mg":
        await page.locator('label:has-text("1.25 mg")').click();
    elif glp1dose == "1.7 mg":
        await page.locator('label:has-text("1.7 mg")').click();
    elif glp1dose == "2 mg":
        await page.locator('label:has-text("2 mg")').click();
    elif glp1dose == "2.4 mg":
        await page.locator('label:has-text("2.4 mg")').click();
    elif glp1dose == "2.5 mg":
        await page.locator('label:has-text("2.5 mg")').click();
    elif glp1dose == "5 mg":
        await page.locator('label:has-text("5 mg")').click();
    elif glp1dose == "7.5 mg":
        await page.locator('label:has-text("7.5 mg")').click();
    elif glp1dose == "10 mg":
        await page.locator('label:has-text("10 mg")').click();
    elif glp1dose == "12.5 mg":
        await page.locator('label:has-text("12.5 mg")').click();
    elif glp1dose == "15 mg":
        await page.locator('label:has-text("15 mg")').click();
    await asyncio.sleep(2)

    return print(f'Last GLP-1 dose: {glp1dose}')

In [101]:
## await glp1_dose('10 mg')

In [102]:
async def glp1_effects(effects):

    if effects == "No":
        await page.locator('label:has-text("No")').click();
    if effects == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Did you have GLP-1 side effects: {effects}')

In [103]:
## await glp1_effects('Yes')

In [104]:
async def glp1_side(glp1side):

    if glp1side == "Nausea":
        await page.locator('label:has-text("Nausea")').click();
    elif glp1side == "Heartburn":
        await page.locator('label:has-text("Heartburn")').click();
    elif glp1side == "Constipation":
        await page.locator('label:has-text("Constipation")').click();
    elif glp1side == "Diarrhea":
        await page.locator('label:has-text("Diarrhea")').click();
    await asyncio.sleep(.5)
    await click_continue()

    return print(f'Chose GLP-1 side effect: {glp1side}')

In [105]:
## await glp1_side('Diarrhea')

In [106]:
async def supplements_choice(supplements):

    if supplements == "No":
        await page.locator('label:has-text("No")').click();
    if supplements == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Are you taking any other meds or supplements: {supplements}')

In [107]:
## await supplements_choice('No')

In [108]:
async def allergies_choice(allergies):

    if allergies == "No":
        await page.locator('label:has-text("No")').click();
    if allergies == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Do you have any allergies: {allergies}')

In [109]:
## await allergies_choice('Yes')

In [110]:
async def allergy_list(allergy):

    target_form = page.locator('//*[@id="searchTerm"]')
    await target_form.fill(allergy)
    await asyncio.sleep(1)
    await page.get_by_role("button").nth(1).click()
    await asyncio.sleep(2)
    await click_continue()

    return print(f'Chose allergy: {allergy}')

In [111]:
## await allergy_list('penicillin')

In [112]:
async def alcohol_choice(alcohol):

    if alcohol == "Never":
        await page.locator('label:has-text("Never")').click();
    elif alcohol == "A few times a year":
        await page.locator('label:has-text("A few times a year")').click();
    elif alcohol == "Once a month":
        await page.locator('label:has-text("Once a month")').click();
    elif alcohol == "Once a week":
        await page.locator('label:has-text("Once a week")').click();
    elif alcohol == "Daily or almost daily":
        await page.locator('label:has-text("Daily or almost daily")').click();
    await asyncio.sleep(2)

    return print(f'Chose alcohol consumption: {alcohol}')

In [113]:
## await alcohol_choice('Once a month')

In [114]:
async def drug_choice(drug):

    if drug == "Cocaine":
        await page.locator('label:has-text("Cocaine")').click();
    elif drug == "Opiates/opioids":
        await page.locator('label:has-text("Opiates/opioids")').click();
    elif drug == "Methamphetamine":
        await page.locator('label:has-text("Methamphetamine")').click();
    elif drug == "Cannabis":
        await page.locator('label:has-text("Cannabis")').click();
    elif drug == "No, none of these":
        await page.locator('label:has-text("No, none of these")').click();
    await asyncio.sleep(.3)
    await click_continue()

    return print(f'Chose drugs taken: {drug}')

In [115]:
## await drug_choice('Cannabis')

In [116]:
async def opioid_choice(opioid):

    if opioid == "No":
        await page.locator('label:has-text("No")').click();
    if opioid == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Taken opioid in last 6 months: {opioid}')

In [117]:
## await opioid_choice('Yes')

In [118]:
async def naltrexone_disclosure():

    await page.locator('label:has-text("I understand")').click();
    await asyncio.sleep(2.2)

In [119]:
async def cannabis_disclosure():

    await page.locator('label:has-text("I understand")').click();
    await asyncio.sleep(2.2)

In [120]:
async def nicotine_choice(nicotine):

    if nicotine == "No":
        await page.locator('label:has-text("No")').click();
    if nicotine == "Yes":
        await page.locator('label:has-text("Yes")').click();
    await asyncio.sleep(2)

    return print(f'Are you using nicotine replacement: {nicotine}')

In [121]:
## await nicotine_choice('Yes')

In [122]:
async def nicotine_disclosure():

    await page.locator('label:has-text("I understand")').click();
    await asyncio.sleep(2.2)

In [123]:
async def bp_choice(bp):

    if bp == "Within the last 18 months":
        await page.locator('label:has-text("Within the last 18 months")').click();
    if bp == "More than 18 months ago":
        await page.locator('label:has-text("More than 18 months ago")').click();
    await asyncio.sleep(2)

    return print(f'Last blood pressure check: {bp}')

In [124]:
## await bp_choice('Within the last 18 months')

In [125]:
async def bp_level(bptop, bpbottom):

    if bptop == "Less than 90":
        await page.locator('label:has-text("Less than 90")').click();
    elif bptop == "90-139":
        await page.locator('label:has-text("90-139")').click();
    elif bptop == "140-169":
        await page.locator('label:has-text("140-169")').click();
    elif bptop == "170 or higher":
        await page.locator('label:has-text("170 or higher")').click();
    await asyncio.sleep(.5)
    if bpbottom == "Less than 50":
        await page.locator('label:has-text("Less than 50")').click();
    elif bpbottom == "50-80":
        await page.locator('label:has-text("50-80")').click();
    elif bpbottom == "81-99":
        await page.locator('label:has-text("81-99")').click();
    elif bpbottom == "100 or higher":
        await page.locator('label:has-text("100 or higher")').click();
    await asyncio.sleep(.3)
    await click_continue()
    await asyncio.sleep(2)

    return print(f'Chose blood pressure: {bptop} over {bpbottom}')

In [126]:
## await bp_level(patient_info['bptop'], patient_info['bpbottom'])

In [127]:
async def craving_frequency(crave_freq):

    if crave_freq == "Nearly every day":
        await page.locator('label:has-text("Nearly every day")').click();
    elif crave_freq == "Multiple times per day":
        await page.locator('label:has-text("Multiple times per day")').click();
    elif crave_freq == "24/7":
        await page.locator('label:has-text("24/7")').click();
    elif crave_freq == "Not sure":
        await page.locator('label:has-text("Not sure")').click();
    await asyncio.sleep(2)

    return print(f'Chose craving frequency: {crave_freq}')

In [128]:
## await craving_frequency(patient_info['crave_freq'])

In [129]:
async def resting_rate(resting):

    if resting == "I don't know":
        await page.locator('label:has-text("I don")').click();
    elif resting == "<70 beats per minute":
        await page.locator('label:has-text("<70 beats per minute")').click();
    elif resting == "70 - 79 beats per minute":
        await page.locator('label:has-text("70 - 79 beats per minute")').click();
    elif resting == "80 - 99 beats per minute":
        await page.locator('label:has-text("80 - 99 beats per minute")').click();
    elif resting == "100 or more beats per minute":
        await page.locator('label:has-text("100 or more beats per minute")').click();
    await asyncio.sleep(2)

    return print(f'Chose resting heart rate: {resting}')

In [130]:
## await resting_rate('80 - 99 beats per minute')

In [131]:
async def med_sides(medsides):

    if medsides == "Never":
        await page.locator('label:has-text("Never")').click();
    elif medsides == "Rarely":
        await page.locator('label:has-text("Rarely")').click();
    elif medsides == "Sometimes":
        await page.locator('label:has-text("Sometimes")').click();
    elif medsides == "Often":
        await page.locator('label:has-text("Often")').click();
    elif medsides == "Always":
        await page.locator('label:has-text("Always")').click();
    await asyncio.sleep(2)

    return print(f'Get side effects from meds: {medsides}')

In [132]:
## await med_sides('Rarely')

In [133]:
async def nausea_rating(nausea):

    if nausea == "1":
        await page.locator('label:has-text("1")').click();
    elif nausea == "2":
        await page.locator('label:has-text("2")').click();
    elif nausea == "3":
        await page.locator('label:has-text("3")').click();
    elif nausea == "4":
        await page.locator('label:has-text("4")').click();
    elif nausea == "5":
        await page.locator('label:has-text("5")').click();
    await asyncio.sleep(2)

    return print(f'How worried about nausea: {nausea}')

In [134]:
## await nausea_rating('2')

In [135]:
async def other_products():

    await page.locator('label:has-text("None of these")').click();
    await asyncio.sleep(.2)
    await click_continue()

In [136]:
async def ethnicity_choice(ethnicity):

    if ethnicity == "South Asian":
        await page.locator('label:has-text("South Asian")').click();
    elif ethnicity == "Asian":
        await page.locator('label:has-text("Asian")').click();
    elif ethnicity == "Black or African American":
        await page.locator('label:has-text("Black or African American")').click();
    elif ethnicity == "Hispanic or Latino":
        await page.locator('label:has-text("Hispanic or Latino")').click();
    elif ethnicity == "Native American":
        await page.locator('label:has-text("Native American")').click();
    elif ethnicity == "Pacific Islander":
        await page.locator('label:has-text("Pacific Islander")').click();
    elif ethnicity == "White":
        await page.locator('label:has-text("White")').click();
    elif ethnicity == "Other":
        await page.locator('label:has-text("Other")').click();
    elif ethnicity == "I prefer not to answer":
        await page.locator('label:has-text("Other")').click();
    await asyncio.sleep(.5)
    await click_continue()

    return print(f'Chose ethnicity: {ethnicity}')

In [137]:
## await ethnicity_choice('White')

In [138]:
async def anything_else():

    await page.locator('label:has-text("No")').click();
    await asyncio.sleep(2)



In [139]:
## await anything_else()

In [140]:
async def glp1_shortage():
    await page.locator('label:has-text("No")').click();
    await asyncio.sleep(2)

In [141]:
async def get_bmi():
    bmi = await page.inner_html('//*[@id="root"]/div/div/div/div/article/div[2]/div[1]/span[2]')
    patient_info['bmi'] = bmi
    await asyncio.sleep(1)
    await click_next()

    return print(f'BMI recorded as {bmi}')

In [142]:
async def get_recommendation():
    recommendation = await page.inner_html('//*[@id="root"]/div[1]/div/div/div/div/article/div[1]/div[3]/div/div/div/div/div/div/div/div[1]/div/div[2]/div/span')
    patient_info['recommendation'] = recommendation

    return print(f'Recommendation is: {recommendation}')

In [293]:
## await get_recommendation()

## Defining the patient_info dictionary


In [143]:
## Patient info dictionary

patient_info = {
## goal = the number of pounds you want to lose (intenger)
'goal' : 25,
## state = the state where medication will be shipped (string)
'state' : 'Kansas',
## dob = date of birth (string - MM-DD-YY)
'dob' : '02-22-1992',
## diet_number = how often tried to lose weight (integer <= 10)
'diet_number' : "1-5",
## crave_freq = how frequently you think about eating (string - "Nearly every day" OR "Multiple times per day" OR "24/7" OR "Not sure)
'crave_freq' : 'Multiple times per day',
## family_history = Does family have a history of weight struggles (string - 'Yes' OR 'No' OR 'I don't know')
'family_history' : 'Yes',
## stress = how frequently stressed (string - 'I rarely feel stressed' OR 'I feel stressed a few times a week' OR 'I feel stressed all the time')
'stress' : 'I feel stressed all the time',
## sleep = number of hours slept nighly (integer)
'sleep' : 8,
## area = where you hold your weight (string - 'Around my stomach or waist' OR 'Hips and thighs' OR 'All over')
'area' : 'All over',
## craving = type of food craved (string - 'Sweet' OR 'Salty' OR 'Both' OR "I don't have cravings")
'craving' : 'Sweet',
## habit = type of eating habit (string - "I'm usually multitasking when I eat" OR "I tend to snack when I'm stressed" OR 'I turn to food for comfort' OR 'I often reward myself with food' OR 'I center my social life around dining out')
'habit' : 'I turn to food for comfort',
## focus = other focuses of your program (string - "Realistic nutrition plan" OR "Movement that works for you" OR 'Getting better sleep' OR 'Building healthier habits' OR 'All of the above') 
'focus' : 'Realistic nutrition plan',
## meaning = what reaching goal would mean to you (string - "Having more energy" OR "Feeling more confident" OR 'Improving overall health' OR 'Feeling better in my body' OR 'Feeling good in clothes') 
'meaning' : 'Having more energy',
## feet and inches = height (string - as separate variables)
'feet' : '5',
'inches' : '9',
## pounds = weight (string)
'pounds' : '190',
## most = Are you currently the heaviest you've ever weighed? (string - 'Yes' or 'No')
'most' : 'Yes',
## sex = sex at birth (string 'Male' OR 'Female')
'sex' : 'Male',
## pregnant = Are you pregnant (string - 'Yes' OR 'No' OR 'Trying')
'pregnant' : 'No',
## breastfeeding = Are you breastfeeding? (string - 'Yes' OR 'No'
'breastfeeding' : 'No',
## identity = Do you identify as male? (string - 'Yes' OR 'No')
'identity' : 'Yes',
## gender = What gender do you identify with (string 'Man' OR 'Woman' OR 'Genderqueer' OR 'Non-binary' OR 'Agender' OR 'Questioning'
'gender' : 'Man',
## disorder = eating disorder symptoms answer (string - "Causing yourself to vomit in order to lose weight" OR "Frequently eating very large amounts of food and feeling like you can't stop eating" OR 'Severely limiting the amount of food you eat due to an intense fear of gaining weight' OR 'None of these')
'disorder' : "None of these",
## diagnosis = eating disorder diagonosis (string - 'Anorexia' OR 'Bulimia' OR 'Binge eating disorder' OR 'None of these')
'diagnosis' : 'Anorexia',
## remission = eating disorder remission status (string - "No, I have been in remission for less than one year" OR "No, I am currently being treated" OR 'Yes, I have been in remission for one year or more')
'remission' : 'No, I am currently being treated',
## purge = purged in last 12 months? (string - 'Yes' OR 'No')
'purge' : 'Yes',
## frequency = frequency of purges in last 3 months (string - "About once a week" OR "About once a month" OR 'Not at all')
'frequency' : 'About once a month',
## psycotherapy = are you currently in psycotherapy (string - 'Yes' OR 'No')
'psycotherapy' : 'No',
## mental = have you been diagnosed mental health condition? (string - 'Yes' OR 'No')
'mental' : 'No',
## condition = what mental health condition you've been diagnosed with (string - 'Depression' OR 'Generalized anxiety' OR 'Bipolar disease (manic depression)' OR 'Panic attack' OR 'Psychiatric hospitalization within the last 3 months' OR 'Borderline personality disorder' OR 'Psychosis' OR 'Schizophrenia or schizoaffective disorder')
'condition' : 'Depression',
## medication = taking mental health medication? (string - 'Yes' OR 'No')
'medication' : 'No',
## medlist = the medication you are taking (string - takes whatever input provided and selects the first option from a dropdown)
'medlist' : 'Sertraline',
## suicidal = are you having suicidal/harm thoughts (string - 'Yes' OR 'No')
'suicidal' : 'No',
## chronic = Do you have chronic or medical diseases (string - 'Yes' OR 'No')
'chronic' : 'No',
## kidney = Do you have any of these conditions (string - 'Acute or chronic kidney disease (CKD)' OR 'Kidney stones' OR 'No, none of these')
'kidney' : 'Kidney stones',
## medical = choose medical conditions (string - long list, see medical_condition function above)
'medical' : 'Diabetes (Type 1)',
## other = diagnosed with other medical conditions (string - 'Yes' OR 'No')
'other' : 'No',
## familymed = family history of other diagnoses (string - multiple options, always going with 'No, None of these' for now)
'familymed' : 'No, none of these',
## cardiac = Has anyone under 65 in family died from cardiac event (string - always 'No' for now)
'cardiac' : 'No',
## surgery = Have you had surgeries in past? (string - always 'No' for now)
'surgery' : 'No',
## glp1 = history of taking glp1 medications (string - "I am currently taking a GLP-1 medication" OR "I have taken a GLP-1 medication in the past but I'm not currently" OR "I have never taken a GLP-1 medication")
'glp1' : "I have never taken a GLP-1 medication",
## glp1med = which GLP-1 med you've taken (string - "Injectable liraglutide" OR "Injectable tirzepatide" OR "Injectable semaglutide")
'glp1med' : "Injectable semaglutide",
## glp1last = when last used GLP-1 (string - "Within the last 2 weeks" OR "2-4 weeks ago" OR "Over 4 weeks ago")
'glp1last' : 'Over 4 weeks ago',
## glp1dose = what was last dose of GLP-1 (string - many options, see glp1_dose function above)
'glp1dose' : '1 mg',
## effects = did you have side effects from GLP-1 (string - 'Yes' OR 'No')
'effects' : 'Yes',
## glp1side = which side effects from GLP-1 (string - 'Nausea' OR 'Heartburn' OR 'Constipation' OR 'Diarrhea')
'glp1side' : 'Heartburn',
## supplements = Are you taking other meds/supplements (string - 'Yes' OR 'No')
'supplements' : 'No',
## allergies = do you have allergies (string - 'Yes' OR 'No')
'allergies' : 'Yes',
## allergy = choosing the allergy you have (string - activates dropdown menu)
'allergy' : 'penicillin',
## alcohol = how often consume 5 or more drinks in one occasion (string - "Never" OR "A few times a year" OR "Once a month" OR "Once a week" OR "Daily or almost daily")
'alcohol' : 'Once a week',
## drug == which drugs taken in past 6 months (string - "Cocaine" OR "Opiates/opioids" OR "Methamphetamine" OR "Cannabis" OR "No, none of these")
'drug' : 'Cannabis',
## opioid == have you taken opioids in the last 6 months (string - "Yes" OR "No")
'opioid' : 'Yes',    
## nicotine == are you taking nicotine replacement (string - 'Yes' OR 'No')
'nicotine' : 'No',
## bp = when last blood pressure check (string - "Within the last 18 months" OR "More than 18 months ago")
'bp' : 'Within the last 18 months',
## bptop = top blood pressure number (string - "Less than 90" OR "90-139" OR "140-169" OR "170 or higher")
'bptop' : '90-139',
## bpbottom = bottom blood pressure number (string - "Less than 50" OR "50-80" OR "81-99" OR "100 or higher")
'bpbottom' : '50-80',
## resting = resting heart rate (string - "I don't know" OR "<70 beats per minute" OR "70 - 90 beats per minute" OR "80 - 99 beats per minute" OR "100 or more beats per minute")
'resting' : "80 - 99 beats per minute",
## medsides = do you get side effects from meds (string - "Never" OR "Rarely" OR "Sometimes" OR "Often" OR "Always")
'medsides' : 'Rarely',
## nausea = how worried about nausea are you ( string - "1" OR "2" OR "3" OR "4" Or "5")
'nausea' : '1',
## ethnicity - choose ethnicity (string "South Asian" OR "Black or African American" OR "Hispanic or Latino" OR "White" OR "Pacific Islander" OR "Other" OR "I prefer not to answer" note: "Asian" isn't currently working)
'ethnicity' : 'White',
    
}

## target = 'pounds' - 'goal' (string)
patient_info['target'] = str((int(patient_info['pounds']) - patient_info['goal']))


## Defining the main scraper functions

In [148]:
## A function that finds what page the form is on, enters the relevant information from patient_info, and proceeds to the next page

async def find_page(patient_info):

    ## pages by number, for pages that have unique urls
    page1 = "https://www.hims.com/c/wm/introduction"
    page2 = "https://www.hims.com/c/wm/your-goal"
    page3 = "https://www.hims.com/c/wm/your-goal-transition"
    page4 = "https://www.hims.com/c/wm/no-of-programs-tried"
    page5 = "https://www.hims.com/c/wm/wl-reassurance"
    page6 = "https://www.hims.com/c/wm/select-state"
    page7 = "https://www.hims.com/c/wm/date-of-birth"
    page8 = "https://www.hims.com/c/wm/good-to-go-transition"
    page9 = "https://www.hims.com/c/wm/any-family-with-weight-struggle"
    page10 = "https://www.hims.com/c/wm/not-fate"
    page11 = "https://www.hims.com/c/wm/daily-life-stress-level"
    page12 = "https://www.hims.com/c/wm/quality-sleep"
    page13 = "https://www.hims.com/c/wm/quality-sleep-transition"
    page14 = "https://www.hims.com/c/wm/weight-hold-sites"
    page15 = "https://www.hims.com/c/wm/hormone-influence"
    page17 = "https://www.hims.com/c/wm/eating-patterns"
    page18 = "https://www.hims.com/c/wm/eating-pattern-program-support"
    page19 = "https://www.hims.com/c/wm/eating-pattern-goal"
    page20 = "https://www.hims.com/c/wm/account"
    page21 = "https://www.hims.com/c/wm/intake-up-next"
    page22 = "https://www.hims.com/c/wm/intake-height-weight"
    page24 = "https://www.hims.com/c/wm/medical-intake?offset=2"
    page25 = "https://www.hims.com/c/wm/what-to-expect"
    page26 = "https://www.hims.com/c/wm/tailored-to-you"
    page27 = "https://www.hims.com/c/wm/expert-support"
    page28 = "https://www.hims.com/c/wm/safety-and-quality"
    final_rec = "https://www.hims.com/c/wm/recommended-product"
    

    numbered_pages = [ page1, page2, page3, page4, page5, page6, page7, page8, page9, page10, page11, page12, page13, page14, page15, page17, page18, page19, page20, page21, page22, page24, page25, page26, page27, page28]
    
    ## page headlines for pages without unique urls
    crave_freq = "How often do you find yourself thinking about what or how much to eat"
    crave_type = "what type of food do you usually go for"
    sex = "What was your sex assigned at birth?"
    pregnant = "Are you currently pregnant or trying to become pregnant?"
    identity = "Do you identify as a man?"
    gender = "What gender do you most identify with? "
    breastfeeding = "Are you currently breastfeeding?"
    disorder = "Have you ever experienced any of these symptoms?"
    diagnosis = "Have you been diagnosed with any of the following conditions?"
    remission = "Have you been in remission from your anorexia or bulimia eating disorder for one year or more?"
    be_aware = "Please be aware that purging (self-induced vomiting with or without the use of laxatives or diuretics/water pills), severely restricting your calories, or falling below a BMI of 18.5 (underweight) at any time while using a weight loss medication can increase your risk of electrolyte imbalance and potential for seizures. If you begin purging or severely restricting at any time while on treatment, please alert our Care Team and your primary care provider so that appropriate modifications to your treatment plan can be made to ensure your safety and continued success on your weight loss journey."
    psychotherapy = "Are you currently receiving psychotherapy treatment?"
    purge = "Have you purged or forced yourself to vomit in order to lose weight within the last 12 months?"
    frequency = "How frequently have you purged or vomited to lose weight over the last 3 months?"
    mental = "Have you been diagnosed with a mental health condition?"
    condition = "Have you been diagnosed with any of the following?"
    medication = "Are you currently taking any mental health medication"
    medlist = "Please list the mental health medication"
    suicidal = "Do you currently have any desire to harm yourself or others?"
    suicide_disclosure = "If you are experiencing suicidal thoughts and need to speak to someone"
    chronic = "Do you have any medical conditions or chronic diseases?"
    kidney = "Do you have a history of any of the following?"
    other = "Are there any other medical conditions you haven't shared with us already?"
    familymed = "Have you or a family member ever been diagnosed with any of the following conditions?"
    cardiac = "Has anyone in your family under the age of 65 passed away unexpectedly due to a possible cardiac event?"
    surgery = "Have you had any surgeries or medical procedures?"
    glp1 = "Are you currently or have you ever taken a GLP-1 medication?"
    glp1med = "Which GLP-1 medication"
    glp1last = "When did you last use"
    glp1dose = "What was your last dose of"
    effects = "Did you experience any severe side effects when initially increasing your GLP-1 dose?"
    glp1side = "What side effects did you experience?"
    supplements = "Do you currently take any medications or supplements?"
    allergies = "Do you have any allergies?"
    allergy = "Please list what you are allergic to"
    alcohol = "alcoholic drinks in one occasion"
    drug = "Have you taken any of the following recreational drugs"
    opioid = "Have you taken any opiates/opioids, such as heroin"
    naltrexone = "Naltrexone works by blocking the effects of opioids"
    cannabis = "It is recommended to avoid using cannabis while taking your medication"
    nicotine = "Are you currently using any nicotine replacement products"
    nic_disclosure = "Using nicotine replacement products while taking"
    bp = "healthcare provider checked your blood pressure"
    resting = "average resting heart rate"
    medsides = "Do you tend to get side effects from new medication"
    nausea = "rate how you feel about potentially experiencing nausea"
    otherprods = "is there another area of your health you’d like to improve"
    ethnicity = "describe your ethnicity"
    target = "What is your goal weight?"
    most = "Is your current weight the most you have ever weighed?"
    anything = "Is there anything else you want a licensed provider to know"
    shortage = "due to supply shortages"
    denial = "not a candidate for treatment through our platform"
    bmi = "Medication may be a good fit"
    preview = "preview your treatment"
    
    named_pages = [sex, pregnant, identity, gender, breastfeeding, disorder, diagnosis, remission, be_aware, psychotherapy, purge, frequency, mental, condition, medication, medlist, suicidal, suicide_disclosure, chronic, kidney, other, familymed, cardiac, surgery, glp1, glp1med, glp1last, glp1dose, effects, glp1side, supplements, allergies, allergy, alcohol, drug, opioid, naltrexone, cannabis, nicotine, nic_disclosure, bp]
    
    ## clicking next on a page without h2 header
    if await page.locator('//*[@id="root"]/div/div/div/div/div/article/div/div/div/p').is_visible():
        await click_next()

    if page.url == final_rec:
        await get_recommendation()
    
    ## identifying the current page by url and performing the correct function
    if page.url in numbered_pages:
        if page.url == page1:
            await click_continue()
        elif page.url == page2:
            await loss_goal(patient_info['goal'])
        elif page.url == page3:
            await click_next()
        elif page.url == page4:
            await weight_programs(patient_info['diet_number'])
        elif page.url == page5:
            await click_next()
        elif page.url == page6:
            await select_state(patient_info['state'])
        elif page.url == page7:
            await fill_birthday(patient_info['dob'])
        elif page.url == page8:
            await click_next()
        elif page.url == page9:
            await family_struggles(patient_info['family_history'])
        elif page.url == page10:
            await click_next()
        elif page.url == page11:
            await daily_stress(patient_info['stress'])
        elif page.url == page12:
            await nightly_sleep(patient_info['sleep'])
        elif page.url == page13:
            await click_next()
        elif page.url == page14:
            await weight_place(patient_info['area'])
        elif page.url == page15:
            await click_next()
        elif page.url == page17:
            await eating_habit(patient_info['habit'])
        elif page.url == page18:
            await other_focus(patient_info['focus'])
        elif page.url == page19:
            await goal_meaning(patient_info['meaning'])
        elif page.url == page20:
            await enter_email()
            await enter_password()
        elif page.url == page21:
            await click_next()
        elif page.url == page22:
            await enter_height(patient_info['feet'], patient_info['inches'])
            await enter_weight(patient_info['pounds'])
            await click_next()
        elif page.url == page24:
            await most_weighed(patient_info['most'])
        elif page.url == page25:
            await click_continue()
        elif page.url == page26:
            await click_next()
        elif page.url == page27:
            await click_next()
        elif page.url == page28:
            await click_next()
    
    else:
        ## identifying the current page's h2 headline and performing the correct function
        page_headline = await page.inner_html('h2')
        if page_headline == pregnant:
            await pregnant_status(patient_info['pregnant'])
        elif crave_type in page_headline:
            await craving_type(patient_info['craving'])
        elif crave_freq in page_headline:
            await craving_frequency(patient_info['crave_freq'])
        elif page_headline == sex:
            await birth_sex(patient_info['sex'])
        elif page_headline == identity:
            await male_identity(patient_info['identity'])
        elif page_headline == gender:
            await gender_choice(patient_info['gender'])
        elif page_headline == breastfeeding:
            await bf_status(patient_info['breastfeeding'])
        elif page_headline == disorder:
            await eating_disorder(patient_info['disorder'])
        elif page_headline == diagnosis:
            await disorder_diagnosis(patient_info['diagnosis'])
        elif page_headline == remission:
            await remission_choice(patient_info['remission'])
        elif page_headline == be_aware:
            await purge_disclosure()
        elif page_headline == psychotherapy:
            await psycotherapy_choice(patient_info['psycotherapy'])
        elif page_headline == purge:
            await purge_choice(patient_info['purge'])
        elif page_headline == frequency:
            await purge_frequency(patient_info['frequency'])
        elif page_headline == mental:
            await mental_diagnosis(patient_info['mental'])
        elif page_headline == condition:
            try:
                await mental_condition(patient_info['condition'])
            except:
                await medical_condition(patient_info['medical'])
        elif medication in page_headline:
            await mental_medication(patient_info['medication'])
        elif medlist in page_headline:
            await choose_med(patient_info['medlist'])
        elif page_headline == suicidal:
            await suicidal_choice(patient_info['suicidal'])
        elif suicide_disclosure in page_headline:
            await suicide_disclosure()
        elif page_headline == chronic:
            await chronic_choice(patient_info['chronic'])
        elif page_headline == kidney:
            await kidney_choice(patient_info['kidney'])
        elif page_headline == other:
            await medical_other(patient_info['other'])
        elif page_headline == familymed:
            await family_medical(patient_info['familymed'])
        elif page_headline == cardiac:
            await cardiac_history(patient_info['cardiac'])
        elif page_headline == surgery:
            await surgery_history(patient_info['cardiac'])
        elif page_headline == glp1:
            await glp1_history(patient_info['glp1'])
        elif glp1med in page_headline:
            await glp1_med(patient_info['glp1med'])
        elif glp1last in page_headline:
            await glp1_last(patient_info['glp1last'])
        elif glp1dose in page_headline:
            await glp1_dose(patient_info['glp1dose'])
        elif page_headline == effects:
            await glp1_effects(patient_info['effects'])
        elif page_headline == glp1side:
            await glp1_side(patient_info['glp1side'])
        elif page_headline == supplements:
            await supplements_choice(patient_info['supplements'])
        elif allergies in page_headline:
            await allergies_choice(patient_info['allergies'])
        elif allergy in page_headline:
            await allergy_list(patient_info['allergy'])
        elif alcohol in page_headline:
            await alcohol_choice(patient_info['alcohol'])
        elif drug in page_headline:
            await drug_choice(patient_info['drug'])
        elif opioid in page_headline:
            await opioid_choice(patient_info['opioid'])
        elif naltrexone in page_headline:
            await naltrexone_disclosure()
        elif cannabis in page_headline:
            await cannabis_disclosure()
        elif nicotine in page_headline:
            await nicotine_choice(patient_info['nicotine'])
        elif nic_disclosure in page_headline:
            await nicotine_disclosure()
        elif bp in page_headline:
            await bp_choice(patient_info['bp'])
        elif resting in page_headline:
            await resting_rate(patient_info['resting'])
        elif medsides in page_headline:
            await med_sides(patient_info['medsides'])
        elif nausea in page_headline:
            await nausea_rating(patient_info['nausea'])
        elif otherprods in page_headline:
            await other_products()
        elif ethnicity in page_headline:
            await ethnicity_choice(patient_info['ethnicity'])
        elif page_headline == target:
            await target_weight(patient_info['target'])
        elif page_headline == most:
            await most_weighed(patient_info['most'])
        elif anything in page_headline:
            await anything_else();
        elif shortage in page_headline:
            await glp1_shortage()
        elif denial in page_headline:
            patient_info['result'] = 'Denied'
        elif bmi in page_headline:
            await get_bmi()
        elif preview in page_headline:
            await click_next()
        elif bplevel in await page.inner_html('//*[@id="Question-bp_top_bottom_separate"]/div/div/div/div[1]/div/p'):
            await bp_level(patient_info['bptop'], patient_info['bpbottom'])
        
    
        
        

In [165]:
## The final executable function that fills out the form

async def hims_search(patient_info):

    ## go to hims weight loss questionnaire 
    url = 'https://www.hims.com/c/wm/introduction'
    await page.goto(url)
    await asyncio.sleep(8)

    while page.url not in ['https://www.hims.com/c/wm/check-eligibility', 'https://www.hims.com/c/wm/recommended-product']:
        await find_page(patient_info)
        await asyncio.sleep(1)
    
    if page.url == 'https://www.hims.com/c/wm/recommended-product':
        await get_recommendation()
        
    if page.url == 'https://www.hims.com/c/wm/check-eligibility':
        await find_page(patient_info)
       
  


## Execute the scraper

In [159]:
## Open the headful Playwright browser

driver, page = await open_browser()

In [ ]:
## Run the scraper

await hims_search(patient_info)

In [158]:
## Close the headful browser

await driver.close()